# Mount St Helens Example in pyGMT

Remember the GMT example from last week. We made a map of Mt St Helens that included a hill-shaded DEM as background with a scale bar added and some GPS stations with text labels. I paste the script below:

    #!/bin/bash

    gmt begin sthelens_map png,pdf
	     #nicer map frame
	     gmt set MAP_FRAME_TYPE plain 

	     #add DEM, Lambert projection, 4inches wide image, default illumination
	     gmt grdimage @earth_relief_01s -R-122.4/-121.95/46.0/46.33 \
                      -JL-122.2/46.15/46.0/46.3/4i -I+d

	     #add map frame, scale bar 
	     gmt coast -Wthin -Ba0.1f0.01 -BWSne -Lg-122.0/46.05+w5k+l+c46.1

	     #add GPS stations
	     gmt plot -St0.2 -Gred -Wthin,red sites.xy

	     #add station names
	     gmt text -F+f8p,Helvetica-Bold,black+jRB  sites.xy
    gmt end
    
Let's rewrite this using [pyGMT](https://www.pygmt.org) in this notebook. I'll try to leverage the python aspects as much as possible, but I also want to show the parallels between the two as pyGMT really is just a wrapper around GMT. The map itself, at its core, is still generated by GMT.

## 1) Setup

Let's start out with importing pygmt (installation is covered in the lab), creating a figure object and changing to a simple map frame.

In [ ]:
import pygmt

#just as with matplotlib, we're making a figure object first
fig = pygmt.Figure()

#nicer map frame
#gmt set MAP_FRAME_TYPE plain 
pygmt.config(MAP_FRAME_TYPE="plain")


I also just want to show the contents of the station file again. The format is  

    longitude latitude site-id
    
as GMT would expect.

In [ ]:
!sort -u -k3 sites.xy | head

## 2) DEM
Let's start out with the DEM, which we can load with [grdimage](https://www.pygmt.org/latest/api/generated/pygmt.Figure.grdimage.html). Note that pyGMT supports the `@` notation for provided data sets from GMT, so we can specify dataset and resolution directly in the `grid` name, without having to open a file first. If these were your own data, they should be an [xarray](https://xarray.pydata.org/en/stable/generated/xarray.DataArray.html#xarray.DataArray) data type, or a string that points to a GMT-style [grd](https://docs.generic-mapping-tools.org/latest/gmt.html#grd-inout-full) file.

I changed the `region` to use the list format that pyGMT supports, although grdimage also allows to use the command line region string.

The [projection](https://www.pygmt.org/latest/projections/index.html) is given as the GMT string. There's really no way around it. You've gotta read the documentation and figure it out. 

Setting the `shading` to `True` turns default hill shading / illumination on and makes this look like terrain. You could also pass GMT strings to this if you wanted to illuminate from a different angle, etc. 

In [ ]:
#add DEM, Lambert projection, 4inches wide image, default illumination
#gmt grdimage @earth_relief_01s -R-122.4/-121.95/46.0/46.33 -JL-122.2/46.15/46.0/46.3/4i -I+d
fig.grdimage(grid="@earth_relief_01s", 
             region=[-122.4, -121.95, 46.0, 46.33],    # or "-122.4/-121.95/46.0/46.33"
             projection="L-122.2/46.15/46.0/46.3/4i",  # 
             shading=True)

#Let's have a look
fig.show()

## 3) Map Frame & Scalebar

For some reason I decided to generate the map frame using the `coast` command in the shell script. That's OK. But let's use the [basemap](https://www.pygmt.org/latest/api/generated/pygmt.Figure.basemap.html) command here. 

For the [frame](https://www.pygmt.org/latest/tutorials/frames.html) we're turning labeling of the major tick marks on along the west and south borders, and make major (`a`) tick marks every 0.1 degrees and secondary / fine (`f`) tick marks every 0.01 degrees. Note that you could do this for x and y axis separately.

The [map-scale](https://docs.generic-mapping-tools.org/6.1/basemap.html?highlight=basemap#l) follows GMT syntax. We want ours centered on (`g`) -122.0 degrees longitude, 46.05 degrees latitude with a length (`w`) of 5 km.


In [ ]:
#add map frame, scale bar 
#gmt coast -Wthin -Ba0.1f0.01 -BWSne -Lg-122.0/46.05+w5k+l+c46.1
fig.basemap(frame=["WSne", "a0.1f0.01"], map_scale="g-122.0/46.05+w5k")
fig.show()

## 4) Adding Location Points

Using the [plot](https://www.pygmt.org/latest/api/generated/pygmt.Figure.plot.html) command, we can provide the `data` in a file as we're doing below, using the `sites.xy` file. But note that you have flexibility here. The data could come from pandas / geopandas DataFrames, etc. It's worth reading the documentation on this. You can also specify the location by handing 1d arrays to keywords `x` and `y`; this would allow to plot every symbol with a different size which can be specified via the `size` keywork. A popular example for this would be plotting earthquakes and scaling the symbol size by magnitude.

As we want the same `style` for all symbols, we use the triangles of 0.2 inch size of the circumscribing circle again, speciftied via the GMT string.

We want the triangle `color` to be red and their outlines to be drawn with a thin and black `pen`.



In [ ]:
#add GPS stations
#gmt plot -St0.2 -Gred -Wthin,red sites.xy
fig.plot(
    data="sites.xy",                   #could be numpy.ndarray or pandas.DataFrame or xarray.Dataset or geopandas.GeoDataFrame
    style="t0.2i",
    color="red",
    pen="thin,black"
)

fig.show()

## 5) Add Station Names

Lastly, we want to add station names which is done with the [text](https://www.pygmt.org/latest/api/generated/pygmt.Figure.text.html) command. Just as with `plot`, we hand it the `textfiles` with the contents to be plotted. We specify the `font` and `justify` the text to have the origin point given in the `sites.xy` file to be at the right bottom. Since this would overlay the text onto the triangle, let's `offset` it by a bit along the x axis.

In [ ]:
#add station names
#gmt text -F+f8p,Helvetica-Bold,black+jRB  sites.xy
fig.text(
    textfiles="sites.xy",
    font="8p,Helvetica-Bold,white",
    justify="RB",
    offset="-.1c/0c",
)
fig.show()

## 6) All In One Cell

There's no reason to have this all in separate code cells other than for demonstration. In fact, everything you did to the `Figure` in prior code cells or the previous run of a code cell remains part of the image. Think of it as a stack of layers to which you can only add and not subtract anything. So, if you're experimenting with font placement etc. this will start looking ugly pretty quickly as all prior attempts would remain visible.

The common way to make these maps would be in a single cell (or python script), as shown below. We're also saving the figure as PDF using [savefig](https://www.pygmt.org/latest/api/generated/pygmt.Figure.savefig.html#pygmt.Figure.savefig); other formats are possible.

In [ ]:
#all together

fig = pygmt.Figure()

#add DEM, Lambert projection, 4inches wide image, default illumination
fig.grdimage(grid="@earth_relief_01s", 
             region="-122.4/-121.95/46.0/46.33", 
             projection="L-122.2/46.15/46.0/46.3/4i", 
             shading=True)

#add map frame, scale bar 
fig.basemap(frame=["WSne", "a0.1f0.01"], 
            map_scale="g-122.0/46.05+w5k+l+c46.1")

#add GPS stations
fig.plot(
    data="sites.xy",                   #could be numpy.ndarray or pandas.DataFrame or xarray.Dataset or geopandas.GeoDataFrame
    style="t0.2i",
    color="red",
    pen="thin,black"
)

#add station names
fig.text(
    textfiles="sites.xy",
    font="8p,Helvetica-Bold,white",
    justify="RB",
    offset="-.1c/0c"
)
#save it 
fig.savefig("sthelens_map_pygmt.pdf")

#show it
fig.show()